In [1]:
import sys

sys.setrecursionlimit(10000)

input_file = "data/input.txt"


UP = (0, -1)
RIGHT = (1, 0)
DOWN = (0, 1)
LEFT = (-1, 0)

def sum_coordinates(c1, c2):
    return (c1[0] + c2[0], c1[1] + c2[1])

def within_bounds(x, y, max_x, max_y):
    return 0 <= x < max_x and 0 <= y < max_y

def valid_direction(direction, char):
    if char == ".":
        return True
    if char == "^" and direction == UP:
        return True
    if char == ">" and direction == RIGHT:
        return True
    if char == "v" and direction == DOWN:
       return True
    if char == "<" and direction == LEFT:
       return True
    return False

def create_graph(lines, validate_slope = False):
    MAX_X = len(lines[0])
    MAX_Y = len(lines)

    edges = {}

    for x in range(MAX_X):
        for y in range(MAX_Y):
            if lines[y][x] == "#":
                continue
            edges[(x, y)] = {}
            for direction in [UP, RIGHT, DOWN, LEFT]:
                neighbor = sum_coordinates((x, y), direction)
                if within_bounds(*neighbor, MAX_X, MAX_Y):
                    nx, ny = neighbor
                    char = lines[ny][nx]
                    if validate_slope and not valid_direction(direction, char):
                        continue
                    if char != "#":
                        edges[(x, y)][(nx, ny)] = 1

    if not validate_slope:
        for x in range(MAX_X):
            for y in range(MAX_Y):
                if (x, y) not in edges:
                    continue
                if len(edges[(x, y)]) == 2:
                    left_neighbor, right_neighbor = edges[(x, y)].keys()
                    l1 = edges[(x, y)][left_neighbor]
                    l2 = edges[(x, y)][right_neighbor]
                    del edges[(x, y)]
                    del edges[left_neighbor][(x, y)]
                    del edges[right_neighbor][(x, y)]
                    edges[left_neighbor][right_neighbor] = l1 + l2
                    edges[right_neighbor][left_neighbor] = l1 + l2

    return edges

def dfs(edges, node, target_node, path, pathlen):
    if node == target_node:
        return pathlen

    best = 0
    x, y = node

    for neighbor in edges[(x, y)]:
        nx, ny = neighbor
        val = edges[(x, y)][neighbor]
        if (nx, ny) in path:
            continue
        new_path = path.copy()
        new_path.add(neighbor)
        res = dfs(edges, (nx, ny), target_node, new_path, pathlen + val)
        best = max(best, res)
    return best

with open(input_file, 'r') as f:
    lines = [l.strip() for l in f.readlines()]

    start_y = 0
    end_y = len(lines) - 1
    start_x = lines[0].find(".")
    end_x = lines[end_y].find(".")
    start_node = (start_x, start_y)
    end_node = (end_x, end_y)

    edges1 = create_graph(lines, True)
    edges2 = create_graph(lines)

    ans1 = dfs(edges1, start_node, end_node, set(start_node), 0)
    ans2 = dfs(edges2, start_node, end_node, set(start_node), 0)

    print(f"{ans1 = }")
    print(f"{ans2 = }")

ans1 = 2178
ans2 = 6486
